In [29]:
import pandas as pd

In [30]:
bookings = pd.read_csv('https://stepik.org/media/attachments/lesson/360344/bookings.csv', sep=";")

In [31]:
bookings_head = bookings.head(7)

###### проверка содержимого датафрейма

In [32]:
bookings.shape

(119390, 21)

In [33]:
bookings.dtypes

Hotel                         object
Is Canceled                    int64
Lead Time                      int64
arrival full date             object
Arrival Date Year              int64
Arrival Date Month            object
Arrival Date Week Number       int64
Arrival Date Day of Month      int64
Stays in Weekend nights        int64
Stays in week nights           int64
stays total nights             int64
Adults                         int64
Children                     float64
Babies                         int64
Meal                          object
Country                       object
Reserved Room Type            object
Assigned room type            object
customer type                 object
Reservation Status            object
Reservation status_date       object
dtype: object

###### Переименование столбцов для дальнейшего удобства в их мспользовании

In [34]:
columns_list_old = list(bookings.columns)

In [35]:
columns_list_new = list(bookings.columns.str.lower())

In [36]:
for i in range(len(columns_list_new)):
    columns_list_new[i] = columns_list_new[i].replace(' ', '_')

In [43]:
columns_name_dict = dict(zip(columns_list_old, columns_list_new))

In [47]:
bookings = bookings.rename(columns=columns_name_dict)

In [49]:
bookings.head()

,hotel,is_canceled,lead_time,arrival_full_date,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,...,adults,children,babies,meal,country,reserved_room_type,assigned_room_type,customer_type,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015-07-01,2015,July,27,1,0,0,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015-07-01,2015,July,27,1,0,0,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015-07-01,2015,July,27,1,0,1,...,1,0.0,0,BB,GBR,A,C,Transient,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015-07-01,2015,July,27,1,0,1,...,1,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015-07-01,2015,July,27,1,0,2,...,2,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-03


###### Поиск места откуда чаще приезжают в отель

In [56]:
bookings \
    .query('is_canceled == 0') \
    .country \
    .value_counts() \
    .head(5)

PRT    21071
GBR     9676
FRA     8481
ESP     6391
DEU     6069
Name: country, dtype: int64

###### сколько в среднем ночуют в разных отелях

In [65]:
bookings \
    .query('hotel == "Resort Hotel" or hotel == "City Hotel"') \
    .groupby('hotel', as_index=False) \
    .agg({'stays_total_nights': 'mean'}) \
    .round(2)

,hotel,stays_total_nights
0,City Hotel,2.98
1,Resort Hotel,4.32


###### Количество забронировавших но не заселившихся в отель

In [77]:
len(bookings.query('reserved_room_type != assigned_room_type'))

14917

In [78]:
len(bookings.loc[bookings.reserved_room_type != bookings.assigned_room_type])

14917

######  Самые популярные месяцы в 2016 и 2017 годах

In [84]:
bookings.loc[bookings.arrival_date_year == 2016].arrival_date_month.value_counts().sort_values(ascending=False)

October      6203
May          5478
April        5428
September    5394
June         5292
August       5063
March        4824
July         4572
November     4454
February     3891
December     3860
January      2248
Name: arrival_date_month, dtype: int64

In [85]:
bookings.loc[bookings.arrival_date_year == 2017].arrival_date_month.value_counts().sort_values(ascending=False)

May         6313
April       5661
June        5647
July        5313
March       4970
August      4925
February    4177
January     3681
Name: arrival_date_month, dtype: int64

In [87]:
bookings.groupby('arrival_date_year').arrival_date_month.agg(pd.Series.mode)

arrival_date_year
2015    September
2016      October
2017          May
Name: arrival_date_month, dtype: object

###### Самые отменияемы месяца в 2015, 2016, 2017 годах

In [90]:
bookings \
    .query('is_canceled == 1 & hotel == "City Hotel"') \
    .groupby('arrival_date_year') \
    .arrival_date_month \
    .agg(pd.Series.mode)

arrival_date_year
2015    September
2016      October
2017          May
Name: arrival_date_month, dtype: object

In [94]:
bookings[['adults', 'children', 'babies']].mean()

adults      1.856403
children    0.103890
babies      0.007949
dtype: float64

In [97]:
bookings['total_kids'] = bookings['children'] + bookings['babies']

In [101]:
bookings.groupby('hotel').agg({'total_kids': 'mean'}).round(2)

,total_kids
hotel,
City Hotel,0.10
Resort Hotel,0.14


###### уровень отказа от бронирования в зависимости от наличия детей

In [105]:
bookings['has_kids'] = bookings['total_kids'] > 0

In [130]:
bookings.query('has_kids == False').groupby('has_kids').is_canceled.sum() / bookings.query('has_kids == False').shape[0]

has_kids
False    0.372213
Name: is_canceled, dtype: float64

In [132]:
(bookings.groupby('has_kids').is_canceled.value_counts(normalize=True) * 100).round(2)

has_kids  is_canceled
False     0              62.78
          1              37.22
True      0              65.08
          1              34.92
Name: is_canceled, dtype: float64